# 01 – Data Ingestion & Exploration

**Project:** StreamSense – Netflix Hit Predictor  
**Goal of this notebook:**
- Load the raw Netflix CSV dataset from DBFS
- Create a managed Delta table: `netflix_raw`
- Perform initial exploratory analysis:
  - Schema & data types
  - Row counts, duplicates
  - Nulls / missing data
  - Basic distributions for key fields (type, rating, release_year, etc.)

This notebook should be safe to re-run end-to-end.

In [0]:
# 1. Configuration

# Update this if your filename/path is different
DATA_PATH = "dbfs:/Volumes/workspace/my_catalog/my_volume/Netflix Dataset.csv"  # e.g. netflix_data.csv or netflix_titles.csv

print(f"Using data from: {DATA_PATH}")

In [0]:
from pyspark.sql import functions as F

# 2. Load raw CSV into Spark DataFrame

df_raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(DATA_PATH)
)

print(f"Row count (raw): {df_raw.count():,}")
df_raw.printSchema()

In [0]:
# 3. Save as a managed Delta table for reuse

table_name = "netflix_raw"

(
    df_raw
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable(table_name)
)

print(f"Saved table: {table_name}")

In [0]:
%sql
SELECT * FROM netflix_raw
LIMIT 10;


In [0]:
from pyspark.sql import functions as F

df = spark.table("netflix_raw")

print(f"Rows: {df.count():,}")
print(f"Columns: {len(df.columns)}")
print(df.columns)

In [0]:
display(df.limit(10))

In [0]:
# 4. Null counts per column

null_counts = (
    df.select([
        F.sum(F.when(F.col(c).isNull() | (F.col(c) == ""), 1).otherwise(0)).alias(c)
        for c in df.columns
    ])
)

display(null_counts)

In [0]:
if "Show_Id" in df.columns:
    dup_count = (
        df.groupBy("Show_Id")
          .count()
          .filter(F.col("count") > 1)
          .count()
    )
    print(f"Duplicate Show_Id values: {dup_count}")
else:
    print("No 'Show_Id' column found – will choose a different key later.")

In [0]:
df_raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(
        "/Volumes/workspace/my_catalog/my_volume/Netflix Dataset.csv"
    )
)

display(df_raw)

In [0]:
for col_name in ["Type", "Rating", "Release_Date", "Country"]:
    if col_name in df.columns:
        print(f"\nValue counts for '{col_name}':")
        display(
            df.groupBy(col_name)
              .count()
              .orderBy(F.col("count").desc())
        )
    else:
        print(f"\nColumn '{col_name}' not found in this dataset.")

## Initial Findings

- **Row count:** `<fill in from output>`
- **Columns:** `<list key ones e.g. type, title, release_year, rating, ...>`
- **Null patterns:**
  - `director` has many nulls
  - `country` partially missing
- **Duplicates:**
  - `show_id` seems unique (if present) / or some duplicates found
- **Interesting distributions:**
  - Majority of titles are Movies vs TV Shows
  - Release years range from X to Y
  - Ratings concentrated around TV-MA / TV-14, etc.

These findings will drive:
- How we define our `is_hit` label
- Which features are most promising for the model

In [0]:
df_raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(
        "/Volumes/workspace/my_catalog/my_volume/Netflix Dataset.csv"
    )
)

df_raw.printSchema()

## 🧾 Notebook Summary – 01_Data_Ingestion_Exploration

**Objective:**  
Ingest the raw Netflix dataset into Databricks, perform an initial exploration of its structure and quality, and prepare it for downstream cleaning and feature engineering.

**Key steps completed:**
- Loaded the Kaggle Netflix dataset (`Netflix Dataset.csv`) from Databricks Volumes using PySpark  
- Verified schema, data types, and record count  
- Conducted basic exploratory analysis (row counts, null checks, value distributions)  
- Identified key columns for modelling (`category`, `rating`, `release_date`, `duration`, `description`, etc.)  
- Saved the ingested dataset as a managed Delta table: **`netflix_raw`**

**Findings:**
- Dataset contains ~[insert row count] titles with 11 columns  
- Columns are all string-typed and require type conversion and standardisation  
- `release_date` and `duration` fields are useful for numeric feature extraction  
- Missing or null values present in fields like `director` and `cast`

**Next steps:**
- Proceed to **`02_feature_engineering_and_label`** to:
  - Standardise column names to snake_case  
  - Parse `release_year` and numeric `duration`  
  - Add derived features (`is_movie`, `is_hit`)  
  - Save a clean, model-ready Delta table (`netflix_clean` / `netflix_model_data`)